In [1]:
import openai
import instructor
import json
import os
import malaya
from pydantic import BaseModel, Field
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed
from unidecode import unidecode
import re
import random

minimum_len = 15

def simple_cleaning(string):
    return re.sub(r'[ ]+', ' ', unidecode(string).replace('\n', ' ').replace('--', ' ').replace('/', ' ')).strip()

/home/husein/3.9/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/husein/3.9/lib/python3.9/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))


In [2]:
openai.api_type = "azure"
openai.api_base = "https://husein-ai7-aiservices-435901645.openai.azure.com/"
openai.api_version = "2023-07-01-preview"
openai.api_key = ''
engine = 'gpt-35-turbo-16k'

In [3]:
def predict(messages):
    try:
        response = openai.ChatCompletion.create(
            engine=engine,
            messages=messages,
            temperature=1.0,
            max_tokens=1024,
            top_p=1.0
        )
        return response.choices[0]['message']['content']
    except Exception as e:
        print(e)
        return None

In [4]:
system = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
system_malay = 'Above is a conversation between a user and an intelligent assistant. You suppose to simulate conversation between the user and the assistant in malay language. Bear in mind your major request is to ask the assistant to generate some material. So you can ask the assistant either to make it more detailed, add more related information, or any other request to improve the generated material. Be creative and diverse in your request. Make the response short and the language casual.'
next_step = 'Above is a conversation between a user and an intelligent assistant. Now suppose you are the user, generate response according to the generated material to continue the conversation.'

In [5]:
!mkdir ultrachat-muftiwp.gov.my
# !rm ultrachat-maktabahalbakri.com/*.json

In [6]:
def ultrachat(row, n = 1):
    if row['question'].lower().startswith('soalan:'):
        question = row['question'].split(':')[1:]
        question = ':'.join(question).strip()
    else:
        question = row['question']
        
    results = [
        {'role': 'context', 'content': row['paragraph']},
        {'role': 'user', 'content': question},
    ]
    if row['question'] is None:
        return
    
    initial = f"""
    {row['paragraph']}

    {row['question']}
    """.strip()
    messages = [
        {'role': 'system', 'content': system_malay},
        {'role': 'user', 'content': initial + ', jawab dalam bahasa melayu'},
    ]
    r = predict(messages)
    results.append({
        'role': 'assistant', 'content': r,
    })
    messages.append({
        'role': 'assistant', 'content': r,
    })
    
    for _ in range(n):
    
        messages_temp = messages + [
            {'role': 'user', 'content': 'Now suppose you are the user and you dont really understand, so say something in malay related to islam fiqh to continue the conversation based on given context.'}
        ]
        r2 = predict(messages_temp)
        if r2 is None:
            break

        results.append({
            'role': 'user', 'content': r2,
        })
        messages.append({
            'role': 'user', 'content': r2 + ', jawab dalam bahasa melayu',
        })

        r = predict(messages)
        if r is None:
            break
        results.append({
            'role': 'assistant', 'content': r,
        })
        messages.append({
            'role': 'assistant', 'content': r,
        })
    
    return results

In [7]:
from glob import glob

files = sorted(glob('question-muftiwp.gov.my/*.json'))
len(files)

3885

In [8]:
def generate(f):
    index = int(os.path.split(f)[1].replace('.json', ''))
    filename = f'ultrachat-muftiwp.gov.my/{index}.json'
    if os.path.exists(filename):
        return
        
    with open(f) as fopen:
        data = json.load(fopen)
    
    try:
        r = ultrachat(data, n = random.randint(1, 5))
        with open(filename, 'w') as fopen:
            json.dump(r, fopen)
    except:
        pass

In [9]:
generate(files[0])

In [10]:
from tqdm import tqdm

max_worker = 3
for i in tqdm(range((len(files) // 4) * 0, (len(files) // 4) * 1, max_worker)):
    b = files[i: i + max_worker]
    
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(generate, f): f for f in b}

        for future in as_completed(futures):
            future.result()

 80%|███████████████████████████████        | 258/324 [2:16:21<34:33, 31.42s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
Invalid value for 'content': expected a string, got null.


 89%|████████████████████████████████    | 289/324 [2:42:18<2:02:33, 210.10s/it]

Request timed out: HTTPSConnectionPool(host='husein-ai7-aiservices-435901645.openai.azure.com', port=443): Read timed out. (read timeout=600)


 90%|███████████████████████████████████▏   | 292/324 [2:43:41<47:11, 88.47s/it]

The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
Invalid value for 'content': expected a string, got null.


100%|███████████████████████████████████████| 324/324 [2:57:09<00:00, 32.81s/it]
